https://www.sciencedirect.com/science/article/pii/S2211812814005318

In [131]:
# import dependencies
%matplotlib inline
#import keras
from keras.models import Sequential
from keras.activations import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import talos as ta
import warnings
warnings.filterwarnings("ignore")

# suppres NumPy arrays scientific notation and round decimals to three places
np.set_printoptions(suppress=True)
np.printoptions(precision=3, suppress=True)

libraries = [np,
             pd,
             sklearn,
             keras,
             ta]
for library in libraries:
    print(library, ' ', library.__version__)


<module 'numpy' from 'C:\\Users\\MacalusoC\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\numpy\\__init__.py'>   1.17.0
<module 'pandas' from 'C:\\Users\\MacalusoC\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\pandas\\__init__.py'>   0.24.2
<module 'sklearn' from 'C:\\Users\\MacalusoC\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\sklearn\\__init__.py'>   0.20.3
<module 'keras' from 'C:\\Users\\MacalusoC\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\keras\\__init__.py'>   2.2.4
<module 'talos' from 'C:\\Users\\MacalusoC\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\talos\\__init__.py'>   0.6.0


In [132]:
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

tensorflow==1.14.0
talos==0.6.0
six==1.12.0
scikit-learn==0.20.3
pandas==0.24.2
numpy==1.17.0


In [133]:
# import training data
train_data = pd.read_csv('Training_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
train_data.drop(columns=['Sr._No.'], inplace=True)
# import validation data
test_data = pd.read_csv('Test_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
test_data.drop(columns=['Sr._No.'], inplace=True)
# segregate a single row for validation
validation_data = test_data.loc[0]
test_data = test_data.iloc[1:]

In [134]:
# create NumPy arrays of training features and labels
train_features = train_data[['MRR(mm3/min)', 'Ra(µm)']].values
train_labels = train_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Training Features:\n\n', train_features)
print('\nTraining Labels:\n\n', train_labels) 

Training Features:

 [[ 306.67    0.36]
 [ 609.76    0.47]
 [ 909.28    0.52]
 [ 582.94    0.8 ]
 [1158.73    0.9 ]
 [1727.36    1.1 ]
 [ 943.34    1.63]
 [1875.96    1.76]
 [2797.84    2.17]
 [ 793.04    0.38]
 [1577.02    0.48]
 [2351.92    0.54]
 [1555.25    0.81]
 [3092.37    0.82]
 [4611.35    0.97]
 [2196.85    1.92]
 [4366.5     1.91]
 [6508.93    2.03]
 [ 875.13    0.29]
 [1735.95    0.37]
 [2582.46    0.38]
 [1745.24    0.82]
 [3461.88    0.79]
 [5149.9     1.11]
 [2549.2     1.75]
 [5055.49    1.89]
 [7518.86    1.82]]

Training Labels:

 [[ 280.        0.0508    0.4   ]
 [ 280.        0.0508    0.8   ]
 [ 280.        0.0508    1.2   ]
 [ 280.        0.1016    0.4   ]
 [ 280.        0.1016    0.8   ]
 [ 280.        0.1016    1.2   ]
 [ 280.        0.1524    0.4   ]
 [ 280.        0.1524    0.8   ]
 [ 280.        0.1524    1.2   ]
 [ 710.        0.0508    0.4   ]
 [ 710.        0.0508    0.8   ]
 [ 710.        0.0508    1.2   ]
 [ 710.        0.1016    0.4   ]
 [ 710.        0

In [135]:
# create NumPy arrays of validation features and labels
test_features = test_data[['MRR(mm3/min)', 'Ra(µm)']].values
test_labels = test_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Testing Features:\n\n', test_features)
print('\nTesting Labels:\n\n', test_labels)

Testing Features:

 [[2020.44    0.29]
 [6054.12    1.87]
 [5329.21    1.54]
 [5197.29    1.45]]

Testing Labels:

 [[1100.      0.05    0.69]
 [1095.      0.15    1.2 ]
 [1097.      0.13    1.14]
 [1094.      0.12    1.15]]


In [136]:
# create NumPy array for validation data
validation_features = validation_data[['MRR(mm3/min)', 'Ra(µm)']].values
validation_labels = validation_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values

# reshape the validation data because it's only one row
validation_features = validation_features.reshape(1, -1)
validation_labels = validation_labels.reshape(1, -1)

print('Validation Features:\n\n', validation_features)
print('\nValidation Labels:\n\n', validation_labels)

Validation Features:

 [[1106.51    0.21]]

Validation Labels:

 [[1100.      0.05    0.4 ]]


In [137]:
# Transforms features by scaling each feature to a given range, -1 and 1 in this case
MinMax_scaler = MinMaxScaler(feature_range=(-1, 1))
train_features_MinMax = MinMax_scaler.fit_transform(train_features)
train_labels_MinMax = MinMax_scaler.fit_transform(train_labels)
test_features_MinMax = MinMax_scaler.fit_transform(test_features)
test_labels_MinMax = MinMax_scaler.fit_transform(test_labels)
validation_features_MinMax = MinMax_scaler.fit_transform(validation_features)
validation_labels_MinMax = MinMax_scaler.fit_transform(validation_labels)

#MinMax_scaler.inverse_transform(validation_labels_MinMax)

In [138]:
# Standardize features by removing the mean and scaling to unit variance
Standard_scaler = StandardScaler()
train_features_Standard = Standard_scaler.fit_transform(train_features)
train_labels_Standard = Standard_scaler.fit_transform(train_labels)
test_features_Standard = Standard_scaler.fit_transform(test_features)
test_labels_Standard = Standard_scaler.fit_transform(test_labels)
validation_features_Standard = Standard_scaler.fit_transform(validation_features)
validation_labels_Standard = Standard_scaler.fit_transform(validation_labels)

# Standard_scaler.inverse_transform(train_features_Standard)

In [139]:
p = {
    'first_neuron': [8, 12, 24],
    'second_neuron': [36, 48, 62, 84],
    'third_neuron': [36, 48, 62, 84],
    'fourth_neuron': [8, 12, 24],
    'batch_size': [10, 20, 30],
    'activation': [relu,
                   softmax,
                   #selu,
                   #softplus,
                   #softsign,
                   tanh,
                   sigmoid,
                   #hard_sigmoid,
                   #exponential,
                   linear],
    'optimizer' : [#'SGD',
                   #'RMSprop',
                   #'Adagrad',
                   #'Adadelta',
                   'Adam',],
                   #'Adamax',
                   #'Nadam',],
    'loss' : [mean_squared_error,
              mean_absolute_error,] 
              #mean_absolute_percentage_error, 
              #mean_squared_logarithmic_error,
              #squared_hinge,hinge,
              #categorical_hinge,
              #logcosh,],
}

In [140]:
def MinMax_ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation=params['activation']))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features_MinMax, train_labels_MinMax,
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0,
                    validation_data=[test_features_MinMax, test_labels_MinMax])
    
    # model output
    return out, model

In [141]:
def Standard_ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation=params['activation']))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features_Standard, train_labels_Standard,
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0,
                    validation_data=[test_features_Standard, test_labels_Standard])
    
    # model output
    return out, model

In [142]:
features_MinMax = np.concatenate((train_features_MinMax, test_features_MinMax), axis=0)
labels_MinMax = np.concatenate((train_labels_MinMax, test_labels_MinMax), axis=0)

features_Standard = np.concatenate((train_features_Standard, test_features_Standard), axis=0)
labels_Standard = np.concatenate((train_labels_Standard, test_labels_Standard), axis=0)

# run the Talos experiment on MinMax data
MinMax_t = ta.Scan(features_MinMax, labels_MinMax, 
            params=p, 
            model=MinMax_ann,
            )

# run the Talos experiment on Standard data
Standard_t = ta.Scan(features_Standard, labels_Standard, 
            params=p, 
            model=Standard_ann,
            )

In [143]:
df = pd.read_csv('MinMax.csv')
df.loc[338]

round_epochs                                                   100
val_loss                                                  0.369342
val_acc                                                          1
loss                                                      0.452959
acc                                                        0.62963
activation                   <function relu at 0x000002540B253C80>
batch_size                                                      20
first_neuron                                                    24
fourth_neuron                                                    8
loss.1           <function mean_squared_error at 0x000002540B22...
optimizer                                                     Adam
second_neuron                                                   84
third_neuron                                                    84
Name: 338, dtype: object

In [144]:
# Define the MinMax model
MinMax_model = Sequential()
MinMax_model.add(Dense(24, input_dim=2, activation='relu'))
MinMax_model.add(Dense(84, activation='relu'))
MinMax_model.add(Dense(84, activation='relu'))
MinMax_model.add(Dense(8, activation='relu'))
MinMax_model.add(Dense(3, activation='linear'))
MinMax_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

MinMax_model.fit(train_features_MinMax,
                 train_labels_MinMax,
                 validation_data=(test_features_MinMax, test_labels_MinMax),
                 batch_size=20,
                 epochs=100)

Train on 27 samples, validate on 4 samples
Epoch 1/100
27/27 [==============================] - ETA: 0s - loss: 0.6351 - acc: 0.450 - 2s 81ms/step - loss: 0.6855 - acc: 0.3704 - val_loss: 0.6527 - val_acc: 0.5000
Epoch 2/100
27/27 [==============================] - ETA: 0s - loss: 0.6795 - acc: 0.600 - 0s 493us/step - loss: 0.6611 - acc: 0.5556 - val_loss: 0.6561 - val_acc: 0.5000
Epoch 3/100
27/27 [==============================] - ETA: 0s - loss: 0.6675 - acc: 0.550 - 0s 525us/step - loss: 0.6499 - acc: 0.5926 - val_loss: 0.6425 - val_acc: 0.5000
Epoch 4/100
27/27 [==============================] - ETA: 0s - loss: 0.6394 - acc: 0.550 - 0s 850us/step - loss: 0.6381 - acc: 0.6296 - val_loss: 0.6189 - val_acc: 0.5000
Epoch 5/100
27/27 [==============================] - ETA: 0s - loss: 0.6067 - acc: 0.650 - 0s 593us/step - loss: 0.6250 - acc: 0.6296 - val_loss: 0.5934 - val_acc: 0.5000
Epoch 6/100
27/27 [==============================] - ETA: 0s - loss: 0.6587 - acc: 0.550 - 0s 865us/ste

27/27 [==============================] - ETA: 0s - loss: 0.5137 - acc: 0.750 - 0s 684us/step - loss: 0.4748 - acc: 0.6667 - val_loss: 0.3579 - val_acc: 1.0000
Epoch 49/100
27/27 [==============================] - ETA: 0s - loss: 0.5064 - acc: 0.650 - 0s 628us/step - loss: 0.4768 - acc: 0.6296 - val_loss: 0.3569 - val_acc: 1.0000
Epoch 50/100
27/27 [==============================] - ETA: 0s - loss: 0.5110 - acc: 0.650 - 0s 544us/step - loss: 0.4752 - acc: 0.6296 - val_loss: 0.3654 - val_acc: 0.7500
Epoch 51/100
27/27 [==============================] - ETA: 0s - loss: 0.4501 - acc: 0.650 - 0s 612us/step - loss: 0.4735 - acc: 0.6667 - val_loss: 0.3744 - val_acc: 0.7500
Epoch 52/100
27/27 [==============================] - ETA: 0s - loss: 0.4807 - acc: 0.600 - 0s 425us/step - loss: 0.4743 - acc: 0.6667 - val_loss: 0.3792 - val_acc: 0.7500
Epoch 53/100
27/27 [==============================] - ETA: 0s - loss: 0.5476 - acc: 0.700 - 0s 448us/step - loss: 0.4739 - acc: 0.6667 - val_loss: 0.3768

Epoch 96/100
27/27 [==============================] - ETA: 0s - loss: 0.4820 - acc: 0.700 - 0s 1ms/step - loss: 0.4624 - acc: 0.6296 - val_loss: 0.3783 - val_acc: 0.5000
Epoch 97/100
27/27 [==============================] - ETA: 0s - loss: 0.4318 - acc: 0.650 - 0s 721us/step - loss: 0.4626 - acc: 0.6296 - val_loss: 0.3782 - val_acc: 0.5000
Epoch 98/100
27/27 [==============================] - ETA: 0s - loss: 0.4680 - acc: 0.700 - 0s 817us/step - loss: 0.4617 - acc: 0.6296 - val_loss: 0.3678 - val_acc: 0.5000
Epoch 99/100
27/27 [==============================] - ETA: 0s - loss: 0.4804 - acc: 0.550 - 0s 847us/step - loss: 0.4630 - acc: 0.6296 - val_loss: 0.3663 - val_acc: 0.5000
Epoch 100/100
27/27 [==============================] - ETA: 0s - loss: 0.4664 - acc: 0.650 - 0s 708us/step - loss: 0.4621 - acc: 0.6296 - val_loss: 0.3795 - val_acc: 0.5000


In [145]:
df = pd.read_csv('Standard.csv')
df.loc[26]

round_epochs                                                   100
val_loss                                                   0.53435
val_acc                                                          1
loss                                                       0.44738
acc                                                       0.666667
activation                 <function linear at 0x000002540B253F28>
batch_size                                                      20
first_neuron                                                    24
fourth_neuron                                                   12
loss.1           <function mean_squared_error at 0x000002540B22...
optimizer                                                     Adam
second_neuron                                                   84
third_neuron                                                    84
Name: 26, dtype: object

In [146]:
# Define the Standard model
Standard_model = Sequential()
Standard_model.add(Dense(24, input_dim=2, activation='relu'))
Standard_model.add(Dense(84, activation='relu'))
Standard_model.add(Dense(84, activation='relu'))
Standard_model.add(Dense(12, activation='relu'))
Standard_model.add(Dense(3, activation='linear'))
Standard_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Standard_model.fit(train_features_Standard,
                   train_labels_Standard,
                   validation_data=(test_features_Standard, test_labels_Standard),
                   batch_size=20,
                   epochs=100)

Train on 27 samples, validate on 4 samples
Epoch 1/100
27/27 [==============================] - ETA: 0s - loss: 1.0344 - acc: 0.300 - 2s 86ms/step - loss: 1.0306 - acc: 0.3333 - val_loss: 0.9941 - val_acc: 0.7500
Epoch 2/100
27/27 [==============================] - ETA: 0s - loss: 0.9641 - acc: 0.600 - 0s 658us/step - loss: 0.9926 - acc: 0.6296 - val_loss: 0.9782 - val_acc: 0.7500
Epoch 3/100
27/27 [==============================] - ETA: 0s - loss: 0.9169 - acc: 0.650 - 0s 447us/step - loss: 0.9711 - acc: 0.6667 - val_loss: 0.9650 - val_acc: 0.7500
Epoch 4/100
27/27 [==============================] - ETA: 0s - loss: 0.9965 - acc: 0.700 - 0s 1ms/step - loss: 0.9582 - acc: 0.6296 - val_loss: 0.9601 - val_acc: 0.7500
Epoch 5/100
27/27 [==============================] - ETA: 0s - loss: 0.8830 - acc: 0.700 - 0s 488us/step - loss: 0.9473 - acc: 0.6667 - val_loss: 0.9634 - val_acc: 0.7500
Epoch 6/100
27/27 [==============================] - ETA: 0s - loss: 0.9165 - acc: 0.750 - 0s 547us/step 

Epoch 49/100
27/27 [==============================] - ETA: 0s - loss: 0.7320 - acc: 0.600 - 0s 1ms/step - loss: 0.6786 - acc: 0.6667 - val_loss: 0.9076 - val_acc: 0.7500
Epoch 50/100
27/27 [==============================] - ETA: 0s - loss: 0.6945 - acc: 0.750 - 0s 725us/step - loss: 0.6775 - acc: 0.6667 - val_loss: 0.9136 - val_acc: 0.7500
Epoch 51/100
27/27 [==============================] - ETA: 0s - loss: 0.6754 - acc: 0.700 - 0s 988us/step - loss: 0.6760 - acc: 0.7037 - val_loss: 0.9198 - val_acc: 0.7500
Epoch 52/100
27/27 [==============================] - ETA: 0s - loss: 0.6737 - acc: 0.800 - 0s 605us/step - loss: 0.6760 - acc: 0.7037 - val_loss: 0.9247 - val_acc: 0.7500
Epoch 53/100
27/27 [==============================] - ETA: 0s - loss: 0.6269 - acc: 0.700 - 0s 635us/step - loss: 0.6751 - acc: 0.7037 - val_loss: 0.9251 - val_acc: 0.7500
Epoch 54/100
27/27 [==============================] - ETA: 0s - loss: 0.6448 - acc: 0.700 - 0s 674us/step - loss: 0.6735 - acc: 0.7037 - val_l

Epoch 97/100
27/27 [==============================] - ETA: 0s - loss: 0.6757 - acc: 0.700 - 0s 838us/step - loss: 0.6607 - acc: 0.7037 - val_loss: 0.9454 - val_acc: 0.7500
Epoch 98/100
27/27 [==============================] - ETA: 0s - loss: 0.5824 - acc: 0.750 - 0s 884us/step - loss: 0.6613 - acc: 0.7037 - val_loss: 0.9434 - val_acc: 0.7500
Epoch 99/100
27/27 [==============================] - ETA: 0s - loss: 0.7126 - acc: 0.700 - 0s 762us/step - loss: 0.6608 - acc: 0.7037 - val_loss: 0.9465 - val_acc: 0.7500
Epoch 100/100
27/27 [==============================] - ETA: 0s - loss: 0.7119 - acc: 0.750 - 0s 909us/step - loss: 0.6589 - acc: 0.7037 - val_loss: 0.9579 - val_acc: 0.7500


In [147]:
# MinMax prediction
prediction = MinMax_model.predict(validation_features_MinMax)
prediction = MinMax_scaler.inverse_transform(prediction)
# compare
MinMax_scaler.inverse_transform(validation_labels_MinMax)
print('MinMax Prediction:\n\n', prediction)
print('\nActual:\n\n', MinMax_scaler.inverse_transform(validation_labels_MinMax))

MinMax Prediction:

 [[1100.5513    0.55      0.9   ]]

Actual:

 [[1100.      0.05    0.4 ]]


In [148]:
# Standard prediction
prediction = Standard_model.predict(validation_features_Standard)
prediction = Standard_scaler.inverse_transform(prediction)
# compare
print('Standard Prediction:\n\n', prediction)
print('\nActual:\n\n', Standard_scaler.inverse_transform(validation_labels_Standard))

Standard Prediction:

 [[1100.2672        0.16419293    0.50217617]]

Actual:

 [[1100.      0.05    0.4 ]]
